<a href="https://colab.research.google.com/github/enkhara/PracticaDBA/blob/master/Georgina_SCRAPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NOTA: Crea una carpeta en tu drive llamada webcrawling para que te funcione la subida a drive sin modificar nombres de ficheros/carpetas

## Instalar scrapy

In [0]:
!pip install scrapy

## Linkear Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Montar Spider Class

In [0]:
import scrapy
import json

class BlogSpider(scrapy.Spider):
    name = 'blogspider'
    # Podeis cambiar la url inicial por otra u otras paginas
    start_urls = ['https://www.barcelona.cat/ca/que-pots-fer-a-bcn/parcs-i-jardins/tots-els-parcs']
    
    # con esto limito no hacer mas de 10 ejecuciones 
    COUNT_MAX = 100
    count = 0

    def parse(self, response):
        # Aqui scrapeamos los datos y los imprimimos a un fichero
        for park in response.css('div#llistat-resultats div.box'):
            park_name = park.css('h3 a ::text').extract_first().strip()
            park_description = park.css('p.intro ::text').extract_first()
            park_address = park.css('div.dades p strong ::text').extract_first()
              
            # Print a un fichero
            print(f"{park_name};{park_description};{park_address}")#, file=filep)

        # Aqui hacemos crawling (con el follow)
        for next_page in response.css('div#mes-activitats a'):
            self.count = self.count + 1
            if (self.count < self.COUNT_MAX):
                yield response.follow(next_page, self.parse)


## Ejecutar el Crawler + Scraper

In [0]:
# Podeis cambiar la extension y el nombre del fichero data.txt
filep = open('/content/drive/My Drive/webcrawling/data.csv', 'w')

from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BlogSpider)
process.start()
filep.close()

## Descargar resultados

In [0]:
from google.colab import files
files.download('/content/drive/My Drive/webcrawling/data.csv')

## Enviar resultados a Google Cloud Storage

In [0]:
# Necesitas saber el id de tu proyecto en la consola cloud de google (esta arriba a la izquierda)
project_id = 'logical-lock-265215'

In [0]:
# Importa las librerias necesarias y autenticas con google cloud
project_id = 'logical-lock-265215'

from google.colab import auth
auth.authenticate_user()

In [0]:
# Ahora nos traemos lo necesario para cloud storage
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

In [0]:
# Ahora necesitas el nombre de tu segmento, para ello crea o busca uno aqui https://console.cloud.google.com/storage/browser
bucket_name = 'enkhara'

# Tambien puedes definir el nombre del fichero
file_name = 'parques.csv'

In [0]:
from google.cloud import storage
storage_client = storage.Client(project_id)
# Create a bucket object for our bucket
bucket = storage_client.get_bucket(bucket_name)
# Create a blob object from the filepath
blob = bucket.blob('csv/parques.csv')

blob.upload_from_filename('/content/drive/My Drive/webcrawling/data.csv')

# # Ahora cogemos el fichero del resultado del scrapping y lo subimos al bucket
# from googleapiclient.http import MediaFileUpload

# media = MediaFileUpload('/content/drive/My Drive/webcrawling/data.csv', 
#                         mimetype='text/plain',
#                         resumable=True)

# request = gcs_service.objects().insert(bucket=bucket_name, 
#                                        name=file_name,
#                                        media_body=media)

# response = None
# while response is None:
#   # _ is a placeholder for a progress object that we ignore.
#   # (Our file is small, so we skip reporting progress.)
#   _, response = request.next_chunk()

# print('Upload completo')

### Cuando la subida haya finalizado, los datos aparecerán en el explorador de datos almacenados de la consola de Cloud de tu proyecto:




In [0]:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)